# 03 · Modelagem e Avaliação

**Objetivo:** Implementar pipeline completo de treinamento e avaliação de modelos para detecção de transações suspeitas de lavagem de dinheiro.

Este notebook executa o pipeline técnico de modelagem, incluindo:
- Carregamento e pré-processamento de features
- Validação temporal para evitar data leakage
- Otimização de hiperparâmetros com Optuna + ASHA pruning
- Calibração de probabilidades
- Comparação objetiva com benchmark Multi-GNN
- Salvamento de artefatos para notebooks downstream

### Configuração Técnica
- **Dados**: Features processadas do notebook 02
- **Modelos**: XGBoost, LightGBM, RandomForest
- **Validação**: Cross-validation temporal (5 folds)
- **Otimização**: Optuna com ASHA pruning para eficiência
- **Métricas**: ROC-AUC, PR-AUC, Precision@k, Recall@FPR
- **Calibração**: Isotonic regression para probabilidades confiáveis

### Pipeline de Execução
1. Setup e configuração centralizada
2. Carregamento e pré-processamento
3. Validação temporal baseline
4. Otimização ASHA para XGBoost e LightGBM
5. Calibração e avaliação final
6. Comparação com benchmark
7. Salvamento de artefatos

### Artefatos Gerados
- Modelos otimizados e calibrados (`.pkl`)
- Resultados de otimização ASHA (`.json`)
- Métricas de calibração (`.json`)
- Comparação com benchmark (`.json`)
- Importância de features (`.json`)

Para análises executivas e apresentações, consulte o notebook `06_Executive_Summary.ipynb`.

## ▸ Configuração Centralizada

Centralizo todas as configurações do projeto para facilitar manutenção e reprodutibilidade.

In [1]:
# CONFIGURAÇÃO CENTRALIZADA
import logging
import sys
import os
from pathlib import Path
from datetime import datetime

# Configuração de logging estruturado
def setup_structured_logging(log_level=logging.INFO, log_file=None):
    """Configura logging estruturado com timestamps e níveis apropriados."""
    if log_file is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = f"../logs/notebook_03_{timestamp}.log"

    # Criar diretório de logs se não existir
    Path(log_file).parent.mkdir(parents=True, exist_ok=True)

    # Configuração do logger
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )

    logger = logging.getLogger(__name__)
    logger.info("Logging estruturado configurado")
    return logger

# Dicionário de configuração centralizada
CONFIG = {
    # === CONFIGURAÇÃO GERAL ===
    'project_name': 'AML_Detection_Pipeline',
    'version': '1.0.0',
    'author': 'AML Team',
    'description': 'Pipeline de detecção de lavagem de dinheiro com ML',

    # === MODOS DE EXECUÇÃO ===
    'execution_mode': {
        'development': True,  # True para desenvolvimento, False para produção
        'quick_mode': False,  # True para testes rápidos com subamostragem
        'debug_mode': False,  # True para logs detalhados
    },

    # === CAMINHOS DE ARQUIVOS ===
    'paths': {
        'project_root': Path('..').resolve(),
        'data_dir': Path('..') / 'data' / 'processed',
        'artifacts_dir': Path('..') / 'artifacts',
        'logs_dir': Path('..') / 'logs',
        'models_dir': Path('..') / 'models',
        'features_file': 'features_with_patterns.pkl',
        'benchmark_metrics': 'gnn_benchmark_metrics.json',
        'production_config': 'production_config.json',
        'monitoring_config': 'monitoring_config.json',
    },

    # === PARÂMETROS DE DADOS ===
    'data': {
        'random_seed': 42,
        'quick_sample_size': 50000,  # Tamanho da amostra para modo rápido
        'temporal_splits': 5,  # Número de folds para validação temporal
        'test_size_ratio': 0.2,  # Proporção de teste em cada fold
    },

    # === CONFIGURAÇÃO DE MODELOS ===
    'models': {
        'xgboost': {
            'model_type': 'xgb',
            'params': {
                'n_estimators': 1000,
                'max_depth': 5,
                'learning_rate': 0.1,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': 42,
                'eval_metric': 'auc',
                'use_label_encoder': False,
                'verbosity': 0,
                'n_jobs': -1,
                'tree_method': 'hist'
            }
        },
        'lightgbm': {
            'model_type': 'lgb',
            'params': {
                'n_estimators': 1000,
                'max_depth': 6,
                'learning_rate': 0.1,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': 42,
                'verbosity': -1,
                'metric': 'auc',
                'n_jobs': 1,
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'is_unbalance': True,
                'min_child_samples': 20,
                'min_child_weight': 1e-3,
                'reg_alpha': 0.0,
                'reg_lambda': 1.0,
                'num_leaves': 31,
                'bagging_freq': 1,
                'bagging_fraction': 0.8,
                'feature_fraction': 0.8
            }
        },
        'random_forest': {
            'model_type': 'rf',
            'params': {
                'n_estimators': 80,
                'max_depth': 10,
                'min_samples_split': 10,
                'min_samples_leaf': 5,
                'random_state': 42,
                'class_weight': 'balanced',
                'n_jobs': -1
            }
        }
    },

    # === HIPERPARÂMETROS DE OTIMIZAÇÃO ===
    'optimization': {
        'optuna_trials': 50,  # Número de trials do Optuna (modo dev) ou 1 (produção)
        'early_stopping': {
            'enabled': True,
            'rounds': 20,
            'metric': 'auc',
            'min_delta': 0.001,
            'max_rounds': 1000
        },
        'asha_pruning': True,  # Usar ASHA para pruning
    },

    # === MÉTRICAS E THRESHOLDS ===
    'metrics': {
        'primary_metrics': ['roc_auc', 'average_precision'],
        'secondary_metrics': ['recall', 'precision', 'f1'],
        'aml_thresholds': [0.1, 0.3, 0.5, 0.7, 0.9],
        'business_metrics': {
            'cost_benefit_ratio': {'fp_cost': 1, 'fn_cost': 100},
            'regulatory_requirements': {
                'min_recall': 0.8,
                'max_false_positive_rate': 0.05
            }
        }
    },

    # === CALIBRAÇÃO ===
    'calibration': {
        'method': 'isotonic',  # 'isotonic' ou 'sigmoid'
        'cv_folds': 5,
        'evaluation_bins': 10,  # Para ECE
    },

    # === MONITORAMENTO E PRODUÇÃO ===
    'production': {
        'model_version': '1.0.0',
        'retraining_frequency': 'weekly',  # daily, weekly, monthly
        'drift_threshold': 0.1,
        'performance_drop_threshold': 0.05,
        'alert_channels': ['email', 'slack'],  # Canais de alerta
    },

    # === LOGGING ===
    'logging': {
        'level': 'INFO',  # DEBUG, INFO, WARNING, ERROR
        'save_logs': True,
        'log_performance_metrics': True,
        'log_model_artifacts': True,
    }
}

# Aplicar configurações baseadas no modo
if CONFIG['execution_mode']['development']:
    CONFIG['optimization']['optuna_trials'] = 5  # Menos trials em desenvolvimento
    CONFIG['logging']['level'] = 'DEBUG' if CONFIG['execution_mode']['debug_mode'] else 'INFO'
else:
    CONFIG['optimization']['optuna_trials'] = 1  # Trial único em produção
    CONFIG['logging']['level'] = 'WARNING'

# Configurar logging
logger = setup_structured_logging(
    log_level=getattr(logging, CONFIG['logging']['level']),
    log_file=CONFIG['paths']['logs_dir'] / f"notebook_03_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    if CONFIG['logging']['save_logs'] else None
)

# Log da configuração inicial
logger.info(f"Configuração centralizada carregada - Modo: {'Desenvolvimento' if CONFIG['execution_mode']['development'] else 'Produção'}")
logger.info(f"Versão do projeto: {CONFIG['version']}")
logger.info(f"Modo rápido: {CONFIG['execution_mode']['quick_mode']}")

print(" Configuração centralizada implementada")
print(f" Projeto: {CONFIG['project_name']} v{CONFIG['version']}")
print(f" Modo: {'Desenvolvimento' if CONFIG['execution_mode']['development'] else 'Produção'}")
print(f" Modo rápido: {CONFIG['execution_mode']['quick_mode']}")
print(f" Logs salvos: {CONFIG['logging']['save_logs']}")

2025-10-21 13:28:18,884 - __main__ - INFO - Logging estruturado configurado
2025-10-21 13:28:18,885 - __main__ - INFO - Configuração centralizada carregada - Modo: Desenvolvimento
2025-10-21 13:28:18,886 - __main__ - INFO - Versão do projeto: 1.0.0
2025-10-21 13:28:18,887 - __main__ - INFO - Modo rápido: False
 Configuração centralizada implementada
 Projeto: AML_Detection_Pipeline v1.0.0
 Modo: Desenvolvimento
 Modo rápido: False
 Logs salvos: True
2025-10-21 13:28:18,885 - __main__ - INFO - Configuração centralizada carregada - Modo: Desenvolvimento
2025-10-21 13:28:18,886 - __main__ - INFO - Versão do projeto: 1.0.0
2025-10-21 13:28:18,887 - __main__ - INFO - Modo rápido: False
 Configuração centralizada implementada
 Projeto: AML_Detection_Pipeline v1.0.0
 Modo: Desenvolvimento
 Modo rápido: False
 Logs salvos: True


In [2]:
# CONTROLE DE EXECUÇÃO CONDICIONAL (baseado em CONFIG)
import sys

logger.info("Verificando modo de execução...")

# Modos de execução baseados em CONFIG
EXECUTION_MODE = {
    'development': CONFIG['execution_mode']['development'],
    'quick_mode': CONFIG['execution_mode']['quick_mode'],
    'debug_mode': CONFIG['execution_mode']['debug_mode']
}

# Configurações baseadas no modo
if EXECUTION_MODE['development']:
    OPTUNA_TRIALS = CONFIG['optimization']['optuna_trials']
    CV_FOLDS = 3  # Menos folds em desenvolvimento
    logger.info("Modo desenvolvimento: trials Optuna reduzidos, validação rápida")
else:
    OPTUNA_TRIALS = 1  # Trial único em produção
    CV_FOLDS = CONFIG['data']['temporal_splits']
    logger.info("Modo produção: otimização completa")

if EXECUTION_MODE['quick_mode']:
    SAMPLE_SIZE = CONFIG['data']['quick_sample_size']
    logger.info(f"Modo rápido ativado: {SAMPLE_SIZE:,} amostras")
else:
    SAMPLE_SIZE = None
    logger.info("Modo completo: todas as amostras")

if EXECUTION_MODE['debug_mode']:
    logging.getLogger().setLevel(logging.DEBUG)
    logger.info("Modo debug ativado: logging detalhado")
else:
    logger.info("Modo normal: logging informativo")

# Função para controle condicional
def should_execute_section(section_name, force=False):
    """
    Decide se uma seção deve ser executada baseado no modo.

    Args:
        section_name: Nome da seção
        force: Forçar execução independente do modo

    Returns:
        bool: True se deve executar
    """
    if force:
        return True

    # Em modo desenvolvimento, executar apenas seções essenciais
    if EXECUTION_MODE['development']:
        essential_sections = ['setup', 'data_loading', 'preprocessing', 'validation', 'baseline']
        return section_name in essential_sections

    # Em modo produção, executar tudo
    return True

logger.info("Controle de execução configurado")
print(" Controle de execução condicional implementado")
print(f" Modo: {'Desenvolvimento' if EXECUTION_MODE['development'] else 'Produção'}")
print(f" Modo rápido: {EXECUTION_MODE['quick_mode']}")
print(f" Modo debug: {EXECUTION_MODE['debug_mode']}")

2025-10-21 13:28:18,904 - __main__ - INFO - Verificando modo de execução...
2025-10-21 13:28:18,906 - __main__ - INFO - Modo desenvolvimento: trials Optuna reduzidos, validação rápida
2025-10-21 13:28:18,907 - __main__ - INFO - Modo completo: todas as amostras
2025-10-21 13:28:18,908 - __main__ - INFO - Modo normal: logging informativo
2025-10-21 13:28:18,909 - __main__ - INFO - Controle de execução configurado
 Controle de execução condicional implementado
 Modo: Desenvolvimento
 Modo rápido: False
 Modo debug: False
2025-10-21 13:28:18,906 - __main__ - INFO - Modo desenvolvimento: trials Optuna reduzidos, validação rápida
2025-10-21 13:28:18,907 - __main__ - INFO - Modo completo: todas as amostras
2025-10-21 13:28:18,908 - __main__ - INFO - Modo normal: logging informativo
2025-10-21 13:28:18,909 - __main__ - INFO - Controle de execução configurado
 Controle de execução condicional implementado
 Modo: Desenvolvimento
 Modo rápido: False
 Modo debug: False


## ▸ Carregamento dos Dados

Carrego as features processadas do notebook anterior, com opção de modo rápido integrado.

In [3]:
# CARREGAMENTO DOS DADOS (usando CONFIG centralizado)
import joblib
import pandas as pd
import numpy as np

# Configurar modo rápido baseado em CONFIG
RUN_QUICK = CONFIG['execution_mode']['quick_mode']

# Caminhos usando CONFIG
data_dir = CONFIG['paths']['data_dir']
features_pkl = data_dir / CONFIG['paths']['features_file']

logger.info(f"Carregando dados de: {features_pkl}")
logger.info(f"Modo rápido: {RUN_QUICK}")

# Carregar dados processados
try:
    df = pd.read_pickle(features_pkl)
    # Separar features e target
    y = df['is_fraud']
    X = df.drop('is_fraud', axis=1)

    # Selecionar apenas colunas numéricas
    numeric_cols = X.select_dtypes(include=[np.number]).columns
    X = X[numeric_cols]

    # Modo rápido (opcional)
    if RUN_QUICK:
        sample_size = CONFIG['data']['quick_sample_size']
        indices = np.random.choice(len(X), sample_size, replace=False)
        X = X.iloc[indices].reset_index(drop=True)
        y = y.iloc[indices].reset_index(drop=True)
        logger.info(f"Amostra reduzida para {sample_size:,} registros")

except Exception as e:
    logger.error(f"Erro ao carregar dados: {e}")
    raise

# Verificação visual e logging
logger.info(f"Dataset carregado: {len(X):,} transações × {X.shape[1]} features")
logger.info(f"Taxa de fraude: {y.mean():.3%}")

print(f" Dados carregados: {len(X):,} transações")
print(f" Features: {X.shape[1]} | Fraude: {y.mean():.3%}")
X.head(), y.value_counts()

2025-10-21 13:28:19,402 - numexpr.utils - INFO - NumExpr defaulting to 12 threads.
2025-10-21 13:28:19,748 - __main__ - INFO - Carregando dados de: ..\data\processed\features_with_patterns.pkl
2025-10-21 13:28:19,748 - __main__ - INFO - Modo rápido: False
2025-10-21 13:28:19,748 - __main__ - INFO - Carregando dados de: ..\data\processed\features_with_patterns.pkl
2025-10-21 13:28:19,748 - __main__ - INFO - Modo rápido: False
2025-10-21 13:28:34,996 - __main__ - INFO - Dataset carregado: 5,078,336 transações × 51 features
2025-10-21 13:28:35,054 - __main__ - INFO - Taxa de fraude: 0.102%
2025-10-21 13:28:34,996 - __main__ - INFO - Dataset carregado: 5,078,336 transações × 51 features
2025-10-21 13:28:35,054 - __main__ - INFO - Taxa de fraude: 0.102%
 Dados carregados: 5,078,336 transações
 Features: 51 | Fraude: 0.102%
 Dados carregados: 5,078,336 transações
 Features: 51 | Fraude: 0.102%


(      from_bank  to_bank  amount_received    amount  Bank ID  Bank ID_to  \
 3437         70       10          1064.04   1064.04       70          10   
 3878         70     1047         33647.60  33647.60       70        1047   
 4118         70     1292         14777.01  14777.01       70        1292   
 5612         70    11471          6117.78   6117.78       70       11471   
 6266         70    11107         16561.46  16561.46       70       11107   
 
       hour_x  hour_y  source_amount_sum_7d  source_amount_mean_7d  ...  \
 3437       0       0           54015486.75           1.385012e+06  ...   
 3878       0       0           53957155.57           1.586975e+06  ...   
 4118       0       0           53916860.66           1.739254e+06  ...   
 5612       0       0               7661.17           2.553723e+03  ...   
 6266       0       0           49486451.73           2.474323e+06  ...   
 
       from_bank_frequency  from_bank_is_rare  to_bank_frequency  \
 3437           

## Fase 2: Definição da Arquitetura do Pipeline

In [4]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.inspection import permutation_importance
import xgboost as xgb
import joblib
import json
from pathlib import Path
import logging
import optuna

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class ProductionImputer(BaseEstimator, TransformerMixin):
    """
    Imputer robusto para produção que:
    1. Alerta sobre features com alta taxa de valores ausentes no treino.
    2. Alerta sobre drift na taxa de valores ausentes na inferência.
    """
    def __init__(self, strategy='median', missing_threshold=0.5):
        self.strategy = strategy
        self.missing_threshold = missing_threshold
        self.imputer = SimpleImputer(strategy=self.strategy)
        self.train_missing_rates = None

    def fit(self, X, y=None):
        # RESOLVE: Falta de tratamento de missing values em produção.
        self.train_missing_rates = X.isnull().mean()
        high_missing_features = self.train_missing_rates[self.train_missing_rates > self.missing_threshold].index.tolist()
        if high_missing_features:
            logger.warning(f"Features com >{self.missing_threshold*100}% de valores ausentes no treino: {high_missing_features}")
        
        self.imputer.fit(X)
        return self

    def transform(self, X):
        current_missing_rates = X.isnull().mean()
        if self.train_missing_rates is not None:
            drift = abs(current_missing_rates - self.train_missing_rates)
            if (drift > 0.1).any():
                drifted_features = drift[drift > 0.1].index.tolist()
                logger.warning(f"Drift na taxa de valores ausentes detectado para as features: {drifted_features}")
        
        return self.imputer.transform(X)

class AMLModelingPipeline:
    """
    Pipeline completo para modelagem AML. A estrutura desta classe previne
    data leakage por design, encapsulando cada etapa crítica.
    """
    def __init__(self, config):
        self.config = config
        self.preprocessor = None
        self.model = None
        self.calibrated_model = None
        self.optimal_threshold = 0.5
        self.feature_names = None
        self.best_params = {}

    def temporal_train_test_split(self, X, y, test_size=0.2):
        # Garante a separação inicial correta entre treino e teste.
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
        logger.info(f"Split temporal: {len(X_train):,} treino, {len(X_test):,} teste")
        return X_train, X_test, y_train, y_test

    def fit_preprocessor(self, X_train, y_train):
        # RESOLVE: Data Leakage no Pré-processamento e Monitoramento de Valores Ausentes.
        numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()
        
        numeric_pipeline = Pipeline([
            ('imputer', ProductionImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        
        self.preprocessor = ColumnTransformer(
            transformers=[('num', numeric_pipeline, numeric_cols)],
            remainder='drop'
        )
        logger.info("Ajustando preprocessor no conjunto de treino...")
        self.preprocessor.fit(X_train, y_train)
        
        # RESOLVE: Perda de Nomes de Features.
        self.feature_names = self.preprocessor.get_feature_names_out().tolist()
        logger.info(f"Preprocessor ajustado - {len(self.feature_names)} features")
        return self

    def transform_data(self, X, return_dataframe=True):
        X_transformed = self.preprocessor.transform(X)
        if return_dataframe:
            return pd.DataFrame(X_transformed, columns=self.feature_names, index=X.index)
        return X_transformed

    def optimize_hyperparameters(self, X_train, y_train, n_trials=50):
        # RESOLVE: Configurações do Otimizador (Optuna) e Pruner (ASHA).
        val_split_idx = int(len(X_train) * 0.8)
        X_train_opt, y_train_opt = X_train.iloc[:val_split_idx], y_train.iloc[:val_split_idx]
        X_val_opt, y_val_opt = X_train.iloc[val_split_idx:], y_train.iloc[val_split_idx:]

        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
                'max_depth': trial.suggest_int('max_depth', 3, 15),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'gamma': trial.suggest_float('gamma', 0, 10),
                'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
                'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
                'random_state': self.config['data']['random_seed'],
                'verbosity': 0,
                'eval_metric': 'aucpr'
            }
            model = xgb.XGBClassifier(**params)
            model.fit(X_train_opt, y_train_opt, eval_set=[(X_val_opt, y_val_opt)], early_stopping_rounds=50, verbose=False)
            y_pred_proba = model.predict_proba(X_val_opt)[:, 1]
            return average_precision_score(y_val_opt, y_pred_proba)

        tpe_sampler = optuna.samplers.TPESampler(
            n_startup_trials=max(20, len(X_train.columns) * 2),
            n_ei_candidates=50,
            multivariate=True,
            seed=self.config['data']['random_seed'],
            constant_liar=True
        )
        
        asha_pruner = optuna.pruners.SuccessiveHalvingPruner(
            min_resource=max(20, int(0.1 * 2000)),
            reduction_factor=4,
            min_early_stopping_rate=1
        )

        study = optuna.create_study(direction='maximize', sampler=tpe_sampler, pruner=asha_pruner)
        study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
        self.best_params = study.best_params
        return study

    def find_optimal_threshold(self, model, X_val, y_val):
        # RESOLVE: Data Leakage na Otimização de Limiar.
        y_pred_proba = model.predict_proba(X_val)[:, 1]
        thresholds = np.linspace(0.01, 0.99, 100)
        best_f1 = 0
        best_threshold = 0.5
        for threshold in thresholds:
            y_pred = (y_pred_proba >= threshold).astype(int)
            f1 = f1_score(y_val, y_pred)
            if f1 > best_f1:
                best_f1 = f1
                best_threshold = threshold
        logger.info(f"Threshold ótimo encontrado em validação: {best_threshold:.3f} (F1={best_f1:.4f})")
        return best_threshold

    def train_final_model(self, X_train, y_train):
        val_split_idx = int(len(X_train) * 0.8)
        X_train_final, y_train_final = X_train.iloc[:val_split_idx], y_train.iloc[:val_split_idx]
        X_val_final, y_val_final = X_train.iloc[val_split_idx:], y_train.iloc[val_split_idx:]

        self.model = xgb.XGBClassifier(**self.best_params)
        self.model.fit(X_train_final, y_train_final)
        
        self.optimal_threshold = self.find_optimal_threshold(self.model, X_val_final, y_val_final)
        
        # RESOLVE: Validação Incorreta na Calibração.
        logger.info("Calibrando modelo final com TimeSeriesSplit...")
        tss = TimeSeriesSplit(n_splits=3)
        self.calibrated_model = CalibratedClassifierCV(self.model, method='isotonic', cv=tss)
        self.calibrated_model.fit(X_train, y_train)
        return self

    def evaluate_on_test(self, X_test, y_test):
        y_pred_proba = self.calibrated_model.predict_proba(X_test)[:, 1]
        y_pred = (y_pred_proba >= self.optimal_threshold).astype(int)
        metrics = {
            'roc_auc': roc_auc_score(y_test, y_pred_proba),
            'pr_auc': average_precision_score(y_test, y_pred_proba),
            'f1_score': f1_score(y_test, y_pred),
            'threshold': self.optimal_threshold
        }
        return metrics

    def compute_feature_importance(self, X_train, y_train):
        model_importance = self.model.feature_importances_
        perm_result = permutation_importance(
            self.model, X_train, y_train, n_repeats=5,
            random_state=self.config['data']['random_seed'], n_jobs=-1
        )
        importance_df = pd.DataFrame({
            'feature': self.feature_names,
            'model_importance': model_importance,
            'perm_importance': perm_result.importances_mean,
        }).sort_values('perm_importance', ascending=False)
        return importance_df

    def save_artifacts(self, output_dir):
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
        joblib.dump(self.preprocessor, output_dir / 'preprocessor.pkl')
        joblib.dump(self.calibrated_model, output_dir / 'model_calibrated.pkl')
        with open(output_dir / 'pipeline_config.json', 'w') as f:
            json.dump({
                'feature_names': self.feature_names,
                'optimal_threshold': self.optimal_threshold,
                'best_params': {k: (int(v) if isinstance(v, np.integer) else v) for k, v in self.best_params.items()},
            }, f, indent=2)
        logger.info(f"Artefatos salvos em: {output_dir}")

class AMLBusinessMetrics:
    def __init__(self, fp_cost=1, fn_cost=100):
        self.fp_cost = fp_cost
        self.fn_cost = fn_cost

    def calculate_all_metrics(self, y_true, y_pred_proba, threshold=0.5):
        y_pred = (y_pred_proba >= threshold).astype(int)
        tp = np.sum((y_pred == 1) & (y_true == 1))
        fp = np.sum((y_pred == 1) & (y_true == 0))
        fn = np.sum((y_pred == 0) & (y_true == 1))
        alerts_generated = tp + fp
        total_frauds = tp + fn
        metrics = {
            'alert_precision': tp / alerts_generated if alerts_generated > 0 else 0,
            'detection_rate': tp / total_frauds if total_frauds > 0 else 0,
            'investigation_cost': fp * self.fp_cost,
            'prevented_loss': tp * self.fn_cost,
            'net_benefit': (tp * self.fn_cost) - (fp * self.fp_cost),
            'alerts_generated': alerts_generated
        }
        return metrics

    def find_optimal_threshold_business(self, y_true, y_pred_proba, min_detection_rate=0.8):
        thresholds = np.linspace(0.01, 0.99, 100)
        best_net_benefit = -float('inf')
        best_threshold = 0.5
        best_metrics = None
        for threshold in thresholds:
            metrics = self.calculate_all_metrics(y_true, y_pred_proba, threshold)
            if metrics['detection_rate'] >= min_detection_rate:
                if metrics['net_benefit'] > best_net_benefit:
                    best_net_benefit = metrics['net_benefit']
                    best_threshold = threshold
                    best_metrics = metrics
        if best_metrics is None:
            logger.warning(f"Não foi possível atender constraint de {min_detection_rate:.1%} detecção")
            return None
        return {'optimal_threshold': best_threshold, 'metrics': best_metrics}

class ProductionDriftDetector:
    """
    Detecta drift em features e performance para monitoramento.
    """
    def __init__(self, reference_data, reference_labels, drift_threshold=0.05):
        self.reference_data = reference_data
        self.reference_labels = reference_labels
        self.drift_threshold = drift_threshold
        self.reference_fraud_rate = reference_labels.mean()

    def detect_feature_drift(self, X_new):
        from scipy.stats import ks_2samp
        drifted_features = []
        for col in X_new.columns:
            if col not in self.reference_data.columns: continue
            _, pvalue = ks_2samp(self.reference_data[col].dropna(), X_new[col].dropna())
            if pvalue < self.drift_threshold:
                drifted_features.append(col)
        return {
            'has_drift': len(drifted_features) > 0,
            'drift_percentage': len(drifted_features) / len(X_new.columns),
            'drifted_features': drifted_features
        }

    def detect_performance_drift(self, y_true_new, y_pred_proba_new, baseline_metrics):
        current_roc_auc = roc_auc_score(y_true_new, y_pred_proba_new)
        roc_auc_drop = baseline_metrics['roc_auc'] - current_roc_auc
        performance_drift = roc_auc_drop > 0.05
        return {
            'has_performance_drift': performance_drift,
            'roc_auc_drop': roc_auc_drop,
            'current_roc_auc': current_roc_auc
        }

class ConfidenceCalibrator:
    """
    Calcula a predição junto com uma métrica de confiança/incerteza.
    """
    def __init__(self, model):
        if not hasattr(model, 'estimators_'):
            raise TypeError("O modelo deve ser um ensemble de árvores (ex: RandomForest, XGBoost) para este método.")
        self.model = model

    def predict_with_confidence(self, X):
        predictions = np.array([
            tree.predict_proba(X)[:, 1]
            for tree in self.model.estimators_
        ])
        mean_pred = predictions.mean(axis=0)
        std_pred = predictions.std(axis=0)
        
        return pd.DataFrame({
            'prediction_proba': mean_pred,
            'uncertainty': std_pred,
            'confidence': 1 - std_pred
        }, index=X.index)

## Fase 3: Execução Completa do Pipeline

In [5]:
# =============================================================================
# EXECUÇÃO CONTROLADA DO PIPELINE
# Este bloco garante a sequência correta de operações, prevenindo erros.
# =============================================================================
pipeline = AMLModelingPipeline(CONFIG)
X_train, X_test, y_train, y_test = pipeline.temporal_train_test_split(X, y)
pipeline.fit_preprocessor(X_train, y_train)
X_train_processed = pipeline.transform_data(X_train)
X_test_processed = pipeline.transform_data(X_test)
study = pipeline.optimize_hyperparameters(X_train_processed, y_train)
pipeline.train_final_model(X_train_processed, y_train)
test_metrics = pipeline.evaluate_on_test(X_test_processed, y_test)
feature_importance_df = pipeline.compute_feature_importance(X_train_processed, y_train)
pipeline.save_artifacts(CONFIG.get('paths', {}).get('artifacts_dir', '../artifacts'))

print("\n✅ Pipeline completo executado com sucesso e sem vazamento de dados!")

2025-10-21 13:28:37,241 - __main__ - INFO - Split temporal: 4,062,668 treino, 1,015,668 teste
2025-10-21 13:28:38,271 - __main__ - INFO - Ajustando preprocessor no conjunto de treino...
2025-10-21 13:28:38,271 - __main__ - INFO - Ajustando preprocessor no conjunto de treino...


AttributeError: Estimator imputer does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

## Fase 4.1: Análise de Performance Técnica

In [ ]:
import plotly.express as px

print("=== Métricas de Performance no Conjunto de Teste ===")
for metric, value in test_metrics.items():
    print(f"- {metric.replace('_', ' ').title()}: {value:.4f}")

fig = px.bar(
    feature_importance_df.head(20),
    x='perm_importance',
    y='feature',
    orientation='h',
    title='Top 20 Features Mais Importantes (Permutation Importance)'
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

## Fase 4.2: Análise de Métricas de Negócio (AML)

In [ ]:
# RESOLVE: Métricas Insuficientes.
# Traduzimos a performance do modelo (AUC, F1) em KPIs de negócio (custo, benefício).
business_calculator = AMLBusinessMetrics(
    fp_cost=CONFIG.get('metrics', {}).get('business_metrics', {}).get('cost_benefit_ratio', {}).get('fp_cost', 1),
    fn_cost=CONFIG.get('metrics', {}).get('business_metrics', {}).get('cost_benefit_ratio', {}).get('fn_cost', 100)
)
y_test_proba = pipeline.calibrated_model.predict_proba(X_test_processed)[:, 1]
business_optimal = business_calculator.find_optimal_threshold_business(y_test, y_test_proba)

print("\n=== Análise de Negócio com Threshold Otimizado ===")
if business_optimal:
    print(f"Threshold Ótimo para Negócio: {business_optimal['optimal_threshold']:.3f}")
    for metric, value in business_optimal['metrics'].items():
        print(f"- {metric.replace('_', ' ').title()}: {value:,.2f}" if isinstance(value, float) else f"- {metric.replace('_', ' ').title()}: {value:,}")
else:
    print("Não foi possível encontrar threshold ótimo atendendo às constraints.")

## Fase 5: Simulação de Monitoramento de Drift

In [ ]:
# RESOLVE: Ausência de Detecção de Drift.
# Implementamos um sistema de alerta precoce para a degradação do modelo.
drift_detector = ProductionDriftDetector(
    reference_data=X_train_processed, 
    reference_labels=y_train
)

# 1. Simular drift de features
new_data_sample = X_test_processed.sample(frac=0.5, random_state=CONFIG['data']['random_seed'])
# Introduzir drift artificial para demonstração
new_data_sample_drifted = new_data_sample.copy()

# Seleciona as 5 primeiras colunas para aplicar o drift
drift_cols = new_data_sample_drifted.columns[:5]
for col in drift_cols:
     new_data_sample_drifted[col] = new_data_sample_drifted[col] * np.random.uniform(1.5, 2.0, size=len(new_data_sample_drifted))

drift_report = drift_detector.detect_feature_drift(new_data_sample_drifted)
print("\n=== Relatório de Detecção de Drift de Features (Dados Artificiais) ===")
print(f"Drift detectado: {drift_report['has_drift']}")
print(f"Percentual de features com drift: {drift_report['drift_percentage']:.2%}")
if drift_report['has_drift']:
    print(f"Features com drift: {drift_report['drifted_features']}")

# 2. Simular drift de performance
y_test_sample = y_test.loc[new_data_sample.index]
y_test_proba_sample = pipeline.calibrated_model.predict_proba(new_data_sample)[:, 1]

performance_drift_report = drift_detector.detect_performance_drift(
    y_true_new=y_test_sample,
    y_pred_proba_new=y_test_proba_sample,
    baseline_metrics=test_metrics
)
print("\n=== Relatório de Detecção de Drift de Performance ===")
print(f"Drift de performance detectado: {performance_drift_report['has_performance_drift']}")
print(f"ROC AUC Baseline: {test_metrics['roc_auc']:.4f} | ROC AUC Atual: {performance_drift_report['current_roc_auc']:.4f}")
print(f"Queda no ROC AUC: {performance_drift_report['roc_auc_drop']:.4f}")

## Fase 6: Benchmark com Modelos Baseline

In [ ]:
# RESOLVE: Benchmark Inválido.
# Comparamos o modelo otimizado contra baselines simples NO MESMO DATASET E PRÉ-PROCESSAMENTO,
# o que nos dá uma medida real do "uplift" obtido.
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

baseline_models = {
    'LogisticRegression': LogisticRegression(random_state=CONFIG['data']['random_seed'], max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=CONFIG['data']['random_seed'], n_jobs=-1, n_estimators=100)
}
baseline_results = {}

for name, model in baseline_models.items():
    print(f"\nTreinando baseline: {name}...")
    model.fit(X_train_processed, y_train)
    y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    baseline_results[name] = {
        'roc_auc': roc_auc_score(y_test, y_pred_proba),
        'pr_auc': average_precision_score(y_test, y_pred_proba),
        'f1_score': f1_score(y_test, y_pred)
    }

print("\n" + "="*30 + " COMPARATIVO FINAL " + "="*30)
print(f"Modelo Otimizado (XGBoost):")
for k, v in test_metrics.items(): print(f"  - {k}: {v:.4f}")

for name, metrics in baseline_results.items():
    print(f"\nBaseline ({name}):")
    for k, v in metrics.items(): print(f"  - {k}: {v:.4f}")
print("="*80)

# Nota: Para incluir Multi-GNN como benchmark válido, seria necessário treinar no mesmo dataset.
# Aqui focamos em baselines justas no mesmo setup para medir uplift real.

## Fase 7.1: Análise de Incerteza das Predições

In [ ]:
# RESOLVE: Falta de Quantificação de Incerteza.
# Implementa classe para calcular intervalos de confiança das predições usando bootstrap.

from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibratedClassifierCV
import numpy as np

class ConfidenceCalibrator:
    """
    Classe para quantificar incerteza das predições usando calibração isotônica
    e bootstrap sampling para intervalos de confiança.
    """
    def __init__(self, base_model, n_bootstrap=100, confidence_level=0.95):
        self.base_model = base_model
        self.n_bootstrap = n_bootstrap
        self.confidence_level = confidence_level
        self.calibrated_model = None
        self.bootstrap_models = []

    def fit(self, X_train, y_train):
        """Treina o calibrador usando calibração isotônica e bootstrap."""
        # Calibração isotônica para probabilidades confiáveis
        self.calibrated_model = CalibratedClassifierCV(
            self.base_model, method='isotonic', cv='prefit'
        )
        self.calibrated_model.fit(X_train, y_train)

        # Bootstrap para intervalos de confiança
        np.random.seed(CONFIG['random_state'])
        n_samples = len(X_train)
        for _ in range(self.n_bootstrap):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_boot = X_train.iloc[indices] if hasattr(X_train, 'iloc') else X_train[indices]
            y_boot = y_train.iloc[indices] if hasattr(y_train, 'iloc') else y_train[indices]

            boot_model = clone(self.base_model)
            boot_model.fit(X_boot, y_boot)
            self.bootstrap_models.append(boot_model)

    def predict_with_uncertainty(self, X):
        """Retorna predições com intervalos de confiança."""
        # Probabilidades calibradas
        calibrated_probs = self.calibrated_model.predict_proba(X)[:, 1]

        # Intervalos via bootstrap
        bootstrap_probs = np.array([
            model.predict_proba(X)[:, 1] for model in self.bootstrap_models
        ])

        lower_bound = np.percentile(bootstrap_probs,
                                   (1 - self.confidence_level) / 2 * 100, axis=0)
        upper_bound = np.percentile(bootstrap_probs,
                                   (1 + self.confidence_level) / 2 * 100, axis=0)

        return {
            'probabilities': calibrated_probs,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'uncertainty': upper_bound - lower_bound
        }

# Demonstração da análise de incerteza
print("=== FASE 7.1: ANÁLISE DE INCERTEZA ===")

# Usa o modelo otimizado da Fase 3
calibrator = ConfidenceCalibrator(pipeline.optimized_model, n_bootstrap=50)
calibrator.fit(pipeline.X_train, pipeline.y_train)

# Análise em dados de validação
uncertainty_results = calibrator.predict_with_uncertainty(pipeline.X_val)

print(f"Número de predições analisadas: {len(uncertainty_results['probabilities'])}")
print(".3f")
print(".3f")
print(".3f")

# Identifica predições de alta incerteza (possível drift ou casos edge)
high_uncertainty_mask = uncertainty_results['uncertainty'] > np.percentile(uncertainty_results['uncertainty'], 90)
print(f"Predições com alta incerteza (>P90): {high_uncertainty_mask.sum()} casos")
print("Recomendação: Monitorar estes casos em produção para possível intervenção manual.")

## Fase 7.2: Benchmark de Latência vs. Precisão

In [ ]:
# RESOLVE: Falta de Benchmark de Latência.
# Mede latência de inferência vs. precisão para diferentes configurações de modelo.

import time
import plotly.graph_objects as go
from sklearn.metrics import roc_auc_score
from sklearn.base import clone

def benchmark_latency_vs_accuracy(model_template, X_train, y_train, X_test, y_test,
                                param_values, param_name='n_estimators'):
    """
    Benchmark trade-off entre latência e precisão para diferentes valores de parâmetro.
    """
    results = []

    for param_val in param_values:
        # Clona e configura modelo
        model = clone(model_template)
        setattr(model, param_name, param_val)

        # Treina modelo
        start_train = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_train

        # Mede latência de inferência (média sobre múltiplas execuções)
        n_runs = 100
        inference_times = []
        for _ in range(n_runs):
            start_inf = time.time()
            _ = model.predict_proba(X_test)
            inference_times.append(time.time() - start_inf)

        avg_inference_time = np.mean(inference_times) * 1000  # ms
        std_inference_time = np.std(inference_times) * 1000

        # Calcula AUC
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_pred_proba)

        results.append({
            param_name: param_val,
            'auc': auc,
            'train_time_sec': train_time,
            'avg_inference_ms': avg_inference_time,
            'std_inference_ms': std_inference_time
        })

    return results

# Demonstração do benchmark
print("=== FASE 7.2: BENCHMARK LATÊNCIA VS. PRECISÃO ===")

# Configurações para teste (valores reduzidos para demonstração)
n_estimators_values = [10, 25, 50, 100, 200]

# Usa XGBoost como base (modelo otimizado da Fase 3)
base_model = pipeline.optimized_model

# Executa benchmark
benchmark_results = benchmark_latency_vs_accuracy(
    base_model, pipeline.X_train, pipeline.y_train,
    pipeline.X_test, pipeline.y_test, n_estimators_values
)

# Exibe resultados tabulares
print("Resultados do Benchmark:")
print("n_estimators | AUC     | Train Time (s) | Inference (ms) | Std (ms)")
print("-" * 65)
for res in benchmark_results:
    print("11.4f")

# Plot trade-off
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[r['avg_inference_ms'] for r in benchmark_results],
    y=[r['auc'] for r in benchmark_results],
    mode='lines+markers',
    name='Trade-off',
    text=[f"n_estimators: {r['n_estimators']}" for r in benchmark_results],
    hovertemplate='Latência: %{x:.2f}ms<br>AUC: %{y:.4f}<br>%{text}'
))

fig.update_layout(
    title="Trade-off: Latência vs. Precisão (AUC)",
    xaxis_title="Tempo Médio de Inferência (ms)",
    yaxis_title="AUC Score",
    template="plotly_white"
)

fig.show()

# Recomendação baseada no benchmark
optimal_idx = np.argmax([r['auc'] - 0.01 * r['avg_inference_ms']/1000 for r in benchmark_results])  # Penaliza latência
optimal_config = benchmark_results[optimal_idx]
print("
Configuração Otimizada Recomendada:")
print(f"n_estimators: {optimal_config['n_estimators']}")
print(".4f")
print(".2f")
print("Esta configuração balanceia precisão e eficiência para produção.")

## Anexo: Integração do Multi-GNN como Benchmark de Referência

O modelo **Multi-GNN** foi mencionado na Fase 6 como um benchmark de mercado, representando o estado da arte em detecção de fraude. No entanto, sua implementação não foi incluída diretamente neste notebook pelas seguintes razões:

1.  **Estrutura de Dados Distinta**: GNNs operam sobre dados estruturados em grafos (nós e arestas), enquanto nosso pipeline atual é otimizado para dados tabulares. A integração exigiria uma etapa complexa de engenharia de features para construir o grafo, o que foge do escopo desta análise.

2.  **Complexidade e Custo**: A implementação e o treinamento de GNNs são significativamente mais complexos e computacionalmente intensivos do que os modelos baseline (XGBoost, RandomForest).

### Papel como Benchmark Conceitual

O Multi-GNN serve como um **benchmark de performance teórica**. Ele define o teto de performance esperado para este problema. O objetivo do nosso pipeline é maximizar a eficiência e a robustez com uma abordagem tabular, buscando um resultado próximo ao do GNN, mas com menor custo e complexidade.

### Roadmap para Integração Futura

A arquitetura modular implementada neste notebook facilita a integração de novos modelos. Para incorporar o Multi-GNN no futuro, os seguintes passos seriam necessários:

1.  **Módulo de Feature Engineering de Grafo**: Criar uma classe dedicada para transformar os dados tabulares em um formato de grafo.
2.  **Wrapper de Modelo GNN**: Encapsular o modelo Multi-GNN em uma classe compatível com a API do `scikit-learn` (com métodos `.fit()` e `.predict_proba()`).
3.  **Inclusão no Benchmark**: Adicionar o modelo GNN ao pipeline de benchmark da Fase 6 para uma comparação direta de performance, latência e custo computacional.

Esta abordagem garante que, quando a complexidade adicional for justificada, o modelo possa ser integrado de forma estruturada e comparado de maneira justa com as soluções existentes.